# Data anomalies – 3NF [Exercise]
© ExploreAI Academy

Database normalisation is a design technique for decoupling table structures to **reduce** data redundancies and anomalies. 

In this train, we will go through a practical example of normalising a database up to the **Third Normal Form**. We will look at some of the data anomalies that can occur in practice and, at the end of the train, reflect on how **3NF** attempts to mitigate them.  

## Learning objectives
In this train, you will learn how to:
* Normalise a database up to the Third Normal Form.
* Learn how to further decompose a 2NF database into more tables to help eliminate transitive dependencies.
* Understand data anomalies and how database normalisation reduces the likelihood of their occurrence.

## Imports and DB connections

> ⚠️ ⚠️ This exercise extends the concepts introduced in the previous one, Data anomalies – 2NF. Therefore, it's important to ensure that you continue using the modified SoftDevEmployees.db database after successfully completing the previous exercise.

> ⚠️ Since the queries here will modify the database, you will have to get a fresh copy of the modified database to redo the code cells.


In [2]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook.
%load_ext sql

In [3]:
## Load the SoftDevEmployees database stored in your local machine. 
# Make sure the file is saved in the same folder as this notebook.
%sql sqlite:///../db/SoftDevEmployees.db

## Data anomalies

Data anomalies are issues that present themselves in poorly structured or denormalised databases. The following are examples of commonly occurring anomalies which you may find: 

 - **Deletion anomaly**: The deletion of a record that leads to the unintentional removal of another required attribute from the database. 
 - **Insertion anomaly**: The inability to insert a record as it requires additional data which may presently not be available.
 - **Update anomaly**: This occurs when we have duplicated data; if we were to update the affected rows and a single row gets missed, this will lead to a data inconsistency.

## Second Normal Form database

Below is the ERD for the **SoftDevEmployees.db** database which contains several tables. Currently, our database is in the **Second Normal Form (2NF)**. Our goal within this train is to transform this database to conform to the **Third Normal Form**.


<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/Practical_Normalization/2NF.png" alt="Second Normal Form">

## The Third Normal Form – 3NF

For the database to be in the Third Normal Form, we require the following conditions to be met:

1. The database needs to be in the Second Normal Form.
2. There should be no **transitive** dependencies on the **primary key**.

"**No transitive dependencies**" simply means that **no non-key attribute should be dependent on another non-key attribute**.

Generally speaking, one's salary is related to their occupation band, i.e. we do not expect a graduate to be earning a similar salary to their senior counterparts. So, in this case, we find ourselves with a transitive relation between **`EmployeeID`**, **`Salary`**, and **`OccupationBand`**. 

The **occupation band** is dependent on the **salary** and the **salary** is dependent on the **employee's id**. So we can say that the **`OccupationBand`** is **transitively dependent** on the primary key (**`EmployeeID`** ) through the **`Salary`** column.

## Converting to the Third Normal Form

To solve this, we need to break up our **Employees** table into two new tables for **Salaries** and **Occupation Bands** as follows:

<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/Practical_Normalization/3NF.png" alt = "Third Normal Form"/>

### Exercise 1

We need to create the required 3NF tables based on the above structure. 

Most of the tables required for the Third Normal Form are already available from our Second Normal Form.

Create a 3NF version of these tables by simply making a copy of the 2NF tables and **selecting all the data** from the applicable tables.

In [ ]:
# Add your code here

### Exercise 2

Create the two new tables: **`Salaries_3NF`** and **`OccupationBands_3NF`** following the structure specified by the ERD above. Keep in mind the **primary** and **foreign keys**.

In [ ]:
# Add your code here

### Exercise 3

Populate the two newly created tables: **`Salaries_3NF`** and **`OccupationBands_3NF`** by copying the relevant data from **`Employees_2NF`** and other joined tables where applicable.

**Note:** Use joins appropriately depending on the foreign key references made when creating the tables to maintain referential integrity.

In [ ]:
# Add your code here

### Exercise 4

By creating separate tables for **Salaries** and **Occupation Bands**, the structure of the **Employees** table was also modified.

Create the 3NF version of the **Employees** table as per the ERD above. Keep in mind the primary and the foreign keys.

In [ ]:
# Add your code here

### Exercise 5

Populate the new **Employees** table by copying the relevant data from **`Employees_2NF`** and other joined tables where applicable.

**Note:** Again, use joins appropriately depending on the foreign key references made when creating the tables to maintain referential integrity.

In [ ]:
# Add your code here

### Checkpoint: Data anomalies

Which data anomalies that were present in our database do you think have been addressed at this point of the normalisation process?

## Solutions

### Exercise 1

In [ ]:
%%sql

DROP TABLE IF EXISTS Departments_3NF;
DROP TABLE IF EXISTS Employee_Department_3NF;
DROP TABLE IF EXISTS Employee_Role_3NF;
DROP TABLE IF EXISTS Roles_3NF;
DROP TABLE IF EXISTS Titles_3NF;

CREATE TABLE Departments_3NF AS
SELECT * FROM Departments_2NF;

CREATE TABLE Employee_Department_3NF AS
SELECT * FROM Employee_Department_2NF;

CREATE TABLE Employee_Role_3NF AS
SELECT * FROM Employee_Role_2NF;

CREATE TABLE Roles_3NF AS
SELECT * FROM Roles_2NF;

CREATE TABLE Titles_3NF AS
SELECT * FROM Titles_2NF;

### Exercise 2

In [ ]:
%%sql

DROP TABLE IF EXISTS Salaries_3NF;
DROP TABLE IF EXISTS OccupationBands_3NF;

CREATE TABLE Salaries_3NF(
    SalaryID INTEGER NOT NULL,
    Salary REAL,
    BandID INTEGER,
    FOREIGN KEY(BandID) REFERENCES OccupationBands (BandID),
    PRIMARY KEY(SalaryID AUTOINCREMENT)
);

CREATE TABLE OccupationBands_3NF (
    BandID INTEGER NOT NULL,
    OccupationBand VARCHAR,
    PRIMARY KEY(BandID AUTOINCREMENT)
);

### Exercise 3

In [ ]:
%%sql

DELETE FROM OccupationBands_3NF;
DELETE FROM Salaries_3NF;

INSERT INTO OccupationBands_3NF (OccupationBand)
SELECT DISTINCT 
    OccupationBand
FROM Employees_2NF;

INSERT INTO Salaries_3NF (Salary,BandID)
SELECT 
    Salary,
    OB.BandID
FROM Employees_2NF AS EMP
JOIN OccupationBands_3NF AS OB ON OB.OccupationBand = EMP.OccupationBand;

### Exercise 4

In [ ]:
%%sql
DROP TABLE IF EXISTS Employees_3NF;

CREATE TABLE Employees_3NF (
    EmployeeID INTEGER NOT NULL,
    Name VARCHAR,
    Surname VARCHAR,
    TitleID INTEGER,
    SalaryID INTEGER,
    FOREIGN KEY(TitleID) REFERENCES Titles_3NF (TitleID)
    FOREIGN KEY(SalaryID) REFERENCES Salaries_3NF (SalaryID)
    PRIMARY KEY(EmployeeID AUTOINCREMENT)
);

### Exercise 5

In [ ]:
%%sql

DELETE FROM Employees_3NF;

INSERT INTO Employees_3NF (Name,Surname,TitleID,SalaryID)
SELECT 
    Name,
    Surname,
    TitleID,
    SalaryID
FROM Employees_2NF AS EMP
JOIN Salaries_3NF AS S ON EMP.Salary = S.Salary;

With our data now in the **Third Normal Form**, we are encouraged  to explore the data schema and to look at the contents of the various tables in order to solidify our understanding of the transformations which have taken place. 

## Conclusion

We have now completed the normalisation process up to the **Third Normal Form (3NF)** where we ensured that all our attributes are solely dependent on the primary key. This way, we were able to address some of the remaining anomalies that had not been removed during the 2NF process:

- **Deletion anomaly**: We have eliminated the deletion anomalies that could occur on the **`OccupationBand`**. If all Graduates were promoted to Juniors, the "*Graduate*" occupation band would still exist in the table – ready to be used when new graduates are hired.

- All the **update** and **insertion** anomalies were catered for in the $2^{nd}$ normal form.

However, it is important to note that it is sometimes not possible to eliminate **all redundancies and data anomalies** – we can **only minimise** them.